In [124]:
import requests , time, random, traceback
from bs4 import BeautifulSoup as bs
import pandas as pd

In [118]:
def getHiddenParams(url):

    # 取得參數

    res0 = requests.get(url)
    soup0 = bs(res0.text,"lxml")

    cookie = soup0.select("input[name='cookietime']")[0]["value"]
    formhash = soup0.select("input[name='formhash']")[0]["value"]
#     print("取得隱藏參數-","cookie:", cookie, "/formhash:", formhash)
    return {"cookie": cookie, "formhash": formhash}

#############################################

def login(cookie, formhash):    
    # 登入
    login_url = "https://www.eyny.com/member.php?mod=logging&action=login&loginsubmit=yes&loginhash=LZbj0&inajax=1"
    login_headers = {
        "Host"      : "www.eyny.com",
        "Origin"    : "https://www.eyny.com",
        "Referer"   : "https://www.eyny.com/member.php?mod=logging&action=login",
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.142 Safari/537.36"
    }

    payload = {
        "formhash"            : formhash,
        "referer"             : "https://www.eyny.com/./",
        "loginfield"          : "username",
        "username"            : "7003un",
        "password"            : "0939771419",
        "questionid"          : "0",
        "answer"              : "",
        "cookietime"          : cookie,
        "g-recaptcha-response": "03AOLTBLTUd1qbWQ38a7xd157mbTo1CKjAR4-X9S3tZdgdNuhZ3pIlWJRhJYc_ozrF38X3DByrbdGjdVr0YnFtL9MOIITlVoKBwsPbsD8bzklK3c5-FBSg9mgJa3JOG19kVnXsyNmSSXl7y8Vt57bzNjtzNohp5pH1YNtdFuqlDOQtIEX4_b3hDvHMPt56bOdd1Xd85FyaLHlconYo4Aia6ZpUGkAr7T2fBSi7Ix8crMUllEqQ2MaIWIUOPxklULMunzIXR2zq8W2rnzdRuwbtEgLbfZqhIq8956uQg1En3xJobMS6RK_0cWGyIoMesNs8351fTSn1i_ZwN3ujwIotX5VymQGafR4GWWqgzFdQAL8PBheU8J0NFBvm6cCbXAdvPeFl0sBXyOjO"
    }

    sess = requests.session()
    res1 = sess.post(login_url, headers = login_headers, data = payload)
    
    return sess


#############################################


def crawlArticleList(sess):
    # 取得1-10頁的連結
    base_url = "https://www.eyny.com/"
    headers={
        "user-agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.142 Safari/537.36"
    }

    D3 = base_url+"forum-1710-1.html"

    resD3 = sess.get(D3, headers = headers)
    soupD3 = bs(resD3.text,"lxml")
    
    page_info = soupD3.select("div.pg a")

    page_link_list = [ base_url + ele["href"] for ele in page_info[1:10] ]
    page_link_list.insert(0,D3)

#     print(len(page_link_list))

    # 取得所有文章列表 1-10頁的所有文章連結
    all_page_article_links = []
    for page in page_link_list:

        headers={
            "user-agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.142 Safari/537.36"
        }

        respages = loginsess.get(page, headers = headers)
        soupD3page = bs(respages.text,"lxml")

        if page == "https://www.eyny.com/forum-1710-1.html":
            for ele in soupD3page.select("div#threadlist th.common a.xst")[3:]:
                all_page_article_links.append(base_url + ele["href"] )
        else:
            for ele in soupD3page.select("div#threadlist th.common a.xst"):
                all_page_article_links.append(base_url + ele["href"] )
    #     [ all_page_article_link.append(base_url + ele["href"] for ele in soupD3page.select("div#threadlist th.common a.xst"))]
    # 為何報錯    
#         print(page, "is done")

    return all_page_article_links 


#############################################

#取得作者資訊
def getAuthorInfo(sess, link):
    headers={
        "user-agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.142 Safari/537.36"
            }
    res_article = sess.get(link, headers = headers)
    soup_article = bs(res_article.text,"lxml")

    divTable = soup_article.select("div#postlist > div")
    # 作者資訊
    author = soup_article.select("div.authi")[0].text.replace("\n","")
    author_post_cnt = divTable[0].select("dl.pil dd")[0].text
    author_score = divTable[0].select("dl.pil dd")[1].text.replace(" 點","")
    author_diving_value = divTable[0].select("dl.pil dd")[2].text.replace(" 米","")
    return {"author": author, 
            "author_post_cnt": author_post_cnt, 
            "author_score": author_score, 
            "author_diving_value": author_diving_value
           }

def getContent(divTable):
    
    

#############################################

#取得文章資訊

def getArticleData(sess,link):

    headers={
            "user-agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.142 Safari/537.36"
        }
    res_article = sess.get(link, headers = headers)
    soup_article = bs(res_article.text,"lxml")

    divTable = soup_article.select("div#postlist > div")

    #貼文資訊     
    title = soup_article.select("div#postlist h1 a")[0].text + soup_article.select("div#postlist h1 a")[1].text
    time = divTable[0].select("div.pi em")[0].text.replace("發表於 ","")
    review = soup_article.select("div#postlist span.xi1")[0].text
    res_cnt = soup_article.select("div#postlist span.xi1")[1].text
    content = divTable[0].select("tr td.t_f")[0].text.replace("\r\n","\n").replace("\n"," ")
    
    reply_data_list = []
    for i in divTable[1:-1]:
        
        reply_author = i.select("div.authi")[0].text.replace("\n","")
        
        if "發表於" in reply_author:
            reply_author = "該用戶已被刪除"
            reply_content = i.select("tr td.t_f")[0].text.replace("\r\n","\n").replace("\n"," ")
            reply_post_cnt = "-"
            reply_score = "-"
            reply_diving_value = "-"
            reply_time = i.select("div.authi")[0].text.replace("\n","").replace("發表於 ","").replace("|只看該作者","")
            
            reply_data={
                "replyer" : reply_author,
                "reply time": reply_time,
                "replyer's post": reply_post_cnt,
                "replyer's score": reply_score,
                "replyer's diving value": reply_diving_value,
                "reply content" :reply_content
                }
            reply_data_list.append(reply_data)
            
        else:
            reply_content = i.select("tr td.t_f")[0].text.replace("\r\n","\n").replace("\n"," ")
            reply_post_cnt = i.select("dl.pil dd")[0].text
            reply_score = i.select("dl.pil dd")[1].text.replace(" 點","")
            reply_diving_value = i.select("dl.pil dd")[2].text.replace(" 米","")
            reply_time = i.select("div.pi em")[0].text
            
            reply_data={
                "replyer" : reply_author,
                "reply time": reply_time,
                "replyer's post": reply_post_cnt,
                "replyer's score": reply_score,
                "replyer's diving value": reply_diving_value,
                "reply content" :reply_content
                }
            reply_data_list.append(reply_data)
     

    
    
    author = getAuthorInfo(loginsess, link)
    articleData ={
        "title"               : title,
        "time"                : time,
        "author"              : author["author"],
        "review"              : review,
        "reply number"        : res_cnt,
        "author post"         : author["author_post_cnt"],
        "author score"        : author["author_score"],
        "author diving value" : author["author_diving_value"],
        "Content"             : content,
        "reply"               : reply_data_list

    }
    return articleData

########################
#存擋
def saveFile(dataList,name):

    df = pd.DataFrame(dataList)
    df = df[["title","time","reply number","review","Content","author","author post","author score","author diving value","reply"]]
    df.to_excel(name,index=False)
    

In [ ]:
def author(sess,link):
    

In [112]:
# step1 取得變數
url = "https://www.eyny.com/member.php?mod=logging&action=login"

hiddenParams = getHiddenParams(url)
print("Step1 取得參數 OK")

# step2 登入
loginsess = login( hiddenParams["cookie"], hiddenParams["formhash"])
print("Step2 登入 OK")


# step3 文章列表
articleList = crawlArticleList(loginsess)
print("step3 文章列表取得 OK")

# step4 取得文章作者資訊
article_list = []
for link in articleList[1:10]:
    try:
        articleInfo = getArticleData(loginsess,link)
        article_list.append(articleInfo)
        print(link, "done")
    except:
        traceback.print_exc()
        print(link, "fail")
print(article_list)

# step5 存檔
saveFile(article_list, "20190819-D3-EVNY.xlsx")


Step1 取得參數 OK
Step2 登入 OK
step3 文章列表取得 OK
https://www.eyny.com/thread-11774863-1-GU7Y04C7.html done
https://www.eyny.com/thread-12229666-1-GU7Y04C7.html done
https://www.eyny.com/thread-12252246-1-GU7Y04C7.html done
https://www.eyny.com/thread-12255492-1-GU7Y04C7.html done
https://www.eyny.com/thread-12290829-1-GU7Y04C7.html done
https://www.eyny.com/thread-11311869-1-GU7Y04C7.html done
https://www.eyny.com/thread-12029273-1-GU7Y04C7.html done
https://www.eyny.com/thread-9317174-1-GU7Y04C7.html done
https://www.eyny.com/thread-9137004-1-GU7Y04C7.html done
[{'title': '[抱怨專區]有人無緣無故被鎖帳號嗎?', 'time': '2018-5-19 06:14 PM', 'author': 'mtf931', 'review': '4879', 'reply number': '16', 'author post': '22', 'author score': '43', 'author diving value': '1040', 'Content': '我弟去網咖玩.然後就被鎖帳號了.', 'reply': [{'replyer': '鍾小剛', 'reply time': '發表於 2018-5-20 01:05 PM', "replyer's post": '1914', "replyer's score": '594', "replyer's diving value": '610', 'reply content': '網咖的電腦可能有按鍵精靈 BZ有公告過是不能用的 有可能是這樣  不然就是令

NameError: name 'pd' is not defined